## 导包
python3

In [1]:
#coding:utf-8
from sklearn import cross_validation
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

/anaconda3/envs/aind/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/envs/aind/lib/python3.6/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/anaconda3/envs/aind/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  Dep

## 读个数据

In [2]:
dir = 'data/'
train = pd.read_table(dir + 'train_20171215.txt',engine='python')
test_A = pd.read_table(dir + 'test_A_20171225.txt',engine='python')
sample_A = pd.read_table(dir + 'sample_A_20171225.txt',engine='python',header=None)

### 瞅瞅

In [3]:
train.head()

,date,day_of_week,brand,cnt
0,1,3,1,20
1,1,3,5,48
2,2,4,1,16
3,2,4,3,20
4,3,5,1,1411


In [4]:
test_A.head()

,date,day_of_week
0,1032,4
1,1033,5
2,1034,6
3,1035,1
4,1036,2


In [5]:
sample_A.head()

,0,1
0,1032,0
1,1033,1
2,1034,2
3,1035,3
4,1036,4



date  	  int   	  日期，经过脱敏，用数字来表示  
  day_of_week  	  int  	  表示星期几  
  brand  	  int  	  汽车品牌  
  cnt  	  int  	  上牌数  

### 测试数据少了一个Brand。这里先丢掉

In [6]:
actions1 = train.groupby(['date','day_of_week'], as_index=False)['cnt'].agg({'count1':np.sum})

In [7]:
actions1.head()

,date,day_of_week,count1
0,1,3,68
1,2,4,36
2,3,5,5565
3,4,6,4966
4,5,7,3346


## 模型筛选

### RNN
rnn只保留cnt信息。
很明显，数据是按一星期为周期变动的，我们先尝试设定窗口大小为7，14，28下rnn的预测效果

In [8]:
import rnn
import rnn_process

In [20]:
dt = actions1.drop('day_of_week',axis=1)
dt.drop('date',axis=1,inplace=True)

In [21]:
dt.head()

,count1
0,68
1,36
2,5565
3,4966
4,3346


In [23]:
window_size = 7
X,y = rnn_process.window_transform_series(dt.values,window_size)

In [25]:
X[:10]

array([[  68,   36, 5565, 4966, 3346, 3396, 4146],
       [  36, 5565, 4966, 3346, 3396, 4146, 3096],
       [5565, 4966, 3346, 3396, 4146, 3096, 2713],
       [4966, 3346, 3396, 4146, 3096, 2713, 2409],
       [3346, 3396, 4146, 3096, 2713, 2409,  275],
       [3396, 4146, 3096, 2713, 2409,  275, 2735],
       [4146, 3096, 2713, 2409,  275, 2735, 2386],
       [3096, 2713, 2409,  275, 2735, 2386, 1901],
       [2713, 2409,  275, 2735, 2386, 1901, 6072],
       [2409,  275, 2735, 2386, 1901, 6072, 2716]])

In [26]:
y[:10]

array([[3096],
       [2713],
       [2409],
       [ 275],
       [2735],
       [2386],
       [1901],
       [6072],
       [2716],
       [ 216]])

#### 切分数据

In [27]:
# split our dataset into training / testing sets
train_test_split = int(np.ceil(2*len(y)/float(3)))   # set the split point

# partition the training set
X_train = X[:train_test_split,:]
y_train = y[:train_test_split]

# keep the last chunk for testing
X_test = X[train_test_split:,:]
y_test = y[train_test_split:]

# NOTE: to use keras's RNN LSTM module our input must be reshaped to [samples, window size, stepsize] 
X_train = np.asarray(np.reshape(X_train, (X_train.shape[0], window_size, 1)))
X_test = np.asarray(np.reshape(X_test, (X_test.shape[0], window_size, 1)))

### 弄模型

In [28]:
#超参数
lr = 0.001
rho =0.9
epsilon=1e-08
decay=0.0

In [29]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import keras

# given - fix random seed - so we can all reproduce the same results on our default time series
np.random.seed(0)


# TODO: implement build_part1_RNN in my_answers.py
model = rnn.build_part1_RNN(window_size)

# build model using keras documentation recommended optimizer initialization
optimizer = keras.optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=dacay)
# compile the model
model.compile(loss='mean_squared_error', optimizer=optimizer)

ModuleNotFoundError: No module named 'keras'

In [ ]:
model.load_weights('weights.hdf5')

In [ ]:
model.fit(X_train, y_train, epochs=2000, batch_size=50, verbose=0)

In [ ]:
model.save_weights('weights.hdf5')

#### 性能

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# print out training and testing errors
training_error = model.evaluate(X_train, y_train, verbose=0)
print('training error = ' + str(training_error))

testing_error = model.evaluate(X_test, y_test, verbose=0)
print('testing error = ' + str(testing_error))

#### 画图

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot original series
plt.plot(dataset,color = 'k')

# plot training set prediction
split_pt = train_test_split + window_size 
plt.plot(np.arange(window_size,split_pt,1),train_predict,color = 'b')

# plot testing set prediction
plt.plot(np.arange(split_pt,split_pt + len(test_predict),1),test_predict,color = 'r')

# pretty up graph
plt.xlabel('day')
plt.ylabel('(normalized) price of Apple stock')
plt.legend(['original series','training fit','testing fit'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
#### 输出结果
result1 = model.predict(df_test_data).astype(int)

### gbdt

In [19]:
df_train_target = actions1['count1'].values
df_train_data = actions1.drop(['count1'],axis = 1).values
df_test_data = test_A.values

In [16]:
# 切分数据（训练集和测试集）
cv = cross_validation.ShuffleSplit(len(df_train_data), n_iter=5, test_size=0.2,random_state=0)

for train, test in cv:    
    gbdt = GradientBoostingRegressor().fit(df_train_data[train], df_train_target[train])
    result1 = gbdt.predict(df_train_data[test])
    print(mean_squared_error(result1,df_train_target[test]))
    print('......')

564745.025164
......
593765.995252
......
720902.365872
......
460706.91051
......
980977.756306
......


In [34]:
gbdt = GradientBoostingRegressor().fit(df_train_data, df_train_target)
result1 = gbdt.predict(df_test_data).astype(int)

## 输出结果

In [31]:
test_A = test_A.drop('day_of_week',axis=1)

ValueError: labels ['day_of_week'] not contained in axis

In [35]:
test_A['result'] = result1

In [36]:
test_A[['date','result']].to_csv('data/result.txt',index=False,header=False,sep='\t')